> reference
- paper: https://arxiv.org/abs/1704.03477
- drawing test: https://magenta.tensorflow.org/assets/sketch_rnn_demo/index.html
- blogs: 
 - https://magenta.tensorflow.org/sketch-rnn-demo
 - https://ai.googleblog.com/2017/04/teaching-machines-to-draw.html
 - https://magenta.tensorflow.org/sketch_rnn
 - https://experiments.withgoogle.com/sketch-rnn-demo
- github repo: https://github.com/tensorflow/magenta/blob/master/magenta/models/sketch_rnn/README.md

# 3. Methodology

### 3.1 Dataset

- QuickDraw라는 데이터셋을 만듬
- pen stroke action을 표현하는 데이터 포멧을 정의함, $(\Delta x, \Delta y, p_1, p_2, p_3)$
 - $\Delta x$: 이전 점으로부터 $x$ 좌표 변화량
 - $\Delta y$: 이전 점으로부터 $y$ 좌표 변화량
 - $p_1$: 펜의 상태로서 펜이 지금 종이에 닿아 있는지를 표시 (1이면 닿아 있다.)
 - $p_2$: 펜이 현재점 이후 뜰 것임을 표시 (1이면 뜰 것이다.)
 - $p_3$: 그림 다 그렸음을 표시 (1이면 다 그렸다.)
 - 예를들어 (3, 2, 1, 1, 0) 이면 이전 좌표에서 $x$축으로 3만큼, $y$축으로 2만큼 움직였고, 현재 펜이 종이에 닿아 있고, 이후 다른 점에서 다시 시작할 것이며, 아직 그림이 끝나지 않았음을 의미

### 3.2 Sketch-RNN

- 이 모델은 Sequence-to-Sequence Variational Autoencoder 이다.


- Forward Encoder RNN은 순방향 스케치 데이터 $S$를 입력으로 받아 $h_{\rightarrow}$를 출력으로 내놓고, Backward Encoder RNN은 역방향 스케치 데이터 $S_{\text{reverse}}$를 입력으로 받아 $h_{\leftarrow}$를 출력으로 내놓는다. 두 출력을 병합한 출력을 $h = [ h_{\rightarrow}; h_{\leftarrow} ]$라 한다.


- 병합된 출력 $h$를 VAE의 중앙 히든 레이어의 입력으로 넣는데, 이 중앙 히든 레이어는 각각 크기가 $N_z$인 $\mu$와 $\sigma$로 구성된다. 이후 $z \sim p(z \mid x)$를 샘플링하는 과정은 VAE의 문법을 따른다.

- Decoder는 **autoregressive RNN???** 으로서 $z$가 주어졌을 때 스케치 데이터를 생성하는 역할을 한다.
 - 첫 decoder RNN의 히든 입력은 $[h_0; c_0] = \tanh(W_z z + b_z)$이다. ($c_0$은 없을수도 있다.) **$c_0$는 뭐지?**
 - $i$번째 decoder RNN에는 $i-1$번째 decoder RNN-GMM-Softmax의 출력 $S_{i-1}$와 잠재 벡터 $z$을 병합한 $x_i$를 입력으로 넣는다. 
 - 첫번째 decoder RNN의 입력 $S_0$는 $(0, 0, 1, 0, 0)$을 사용한다.
 - $i$번째 decoder RNN-GMM-Softmax의 출력 $S_i$는 $i$ 번째 그릴 점에 대한 확률 분포를 의미한다.

- decoder RNN의 출력 $y_i$는 $(\Delta x, \Delta y)$를 $5M+M$개 차원 벡터로, $(p_1, p_2, p_3)$를 3개 차원 벡터로 표현한다. 
 - 우선 $(\Delta x, \Delta y)$를 $M$개의 bivariate normal dist의 mixture로 표현한다. 이는 3번 수식에 해당하며, $M$개의 bivariate normal dist를 표현하기 위해 $5M$개의 파라미터가 필요하며 mixture weight $\prod$를 표현하기 위해 $M$개의 파라미터가 필요하다.

- 다음으로 $(p_1, p_2, p_3)$를 각각 $(q_1, q_2, q_3)$라는 categorical 변수로 표현하며 $q_1 + q_2 + q_3 = 1$이라는 제약조건이 있다.
 - 결국 decoder RNN은 $[S_{i-1}; z]$를 입력으로, $[h_{i-1}; c_{i-1}]$을 히든 입력으로 받아 히든 출력 $h_i$와 셀 상태 $c_i$를 만들고, 히든 출력은 다음 decoder RNN으로 보내고, $y_i = W_y h_i + b_y, ~ y_i \in \mathbb{R}^{6M + 3}$는 GMM-Softmax 레이어로 보낸다.
 
 - decoder RNN의 출력 $y_i$는 길이가 $6M + 3$인 벡터인데, 앞부분의 $6M$만큼은 $(\Delta x, \Delta y)$ 분포의 파라미터로 사용되고, 뒷부분 3만큼은 $(q_1, q_2, q_3)$를 표현하는데 사용된다. decoder RNN의 출력은 $\hat{y}$이고 그 형태는 아래와 같다.
 $$\big[ (\hat{\Pi}_1, \mu_x, \mu_y, \hat{\sigma}_x, \hat{\sigma}_y, \hat{\rho}_{xy})_1, \dots (\hat{\Pi}_1, \mu_x, \mu_y, \hat{\sigma}_x, \hat{\sigma}_y, \hat{\rho}_{xy})_M,  ~ (\hat{q}_1, \hat{q}_2, \hat{q}_3) \big] = y_i$$
 - $y_i$가 GMM-softmax의 입력으로 사용되기 전에 수식 6과 같이 표준편차로 사용되는 파라미터가 0보다 크게 하기 위한 장치가 들어간다. 또한 수식 (7)과 같이 logit $\hat{q}_k$를 확률 값 $q_k$로 변환하는 과정도 포함된다.
 - 또한 수식 (7)과 같이 mixture weight 벡터를 결정하기 위해 $\hat{\Pi}$를 softmax로 변환하여 categorical 확률 분포를 만든다. 이것을 이용해 실제 hot-encoding 형태의 mixture weight을 결정한다.

- 이 모델을 학습시킬 때 어려운 점은 3가지 펜의 움직임 $p_1, p_2, p_3$에 대한 확률이 매우 불균형하다는 것이다. $p_1$에 대한 확률은 $p_2$에 대한 확률보다 월등히 높으며, $p_3$ 이벤트는 그림을 그리는 동한 단 한번만 발생한다. 이런 상황에 대한 접근법으로 $p_1, p_2, p_3$ 각각의 loss에 서로 다른 가중치를 부여하는 것이다. 예를들어 (1, 10, 100)와 같은 가중치를 사용한다면, 그림을 그리는 동안 한번만 발생하여 전체 loss에 별 영향을 주지 못하는 $p_3$에 대한 loss가 적절히 보정된다.
 - 하지만 지금 다루려는 데이터는 다양한 종류의 이미지를 다뤄야하기 때문에 이 접근접이 적절하지 않다는걸 확인할 수 있었다.

- 대신 위 방법을 조금 변형했는데, 모든 스케치 데이터의 길이를 $N_\max$(training set 각 그림의 길이중 최대값)가 되도록 가공했다. 대부분 스케치 시퀀스의 길이가 $N_\max$보다 작기 때문에, 실제 $S$ 시퀀스 종료부터 $N_\max$까지 (0, 0, 0, 0, 1)로 체워 넣었다.

- 학습 후 스케치를 생성하는 과정은 우선 (첫 시작점을 생성하는 부분은 명확히 나와 있지 않음) 첫 시작점 $S^{'}_1$를 생성하고, 이를 인코더의 입력으로 넣어 GMM-softmax에서 두번째 점 $S^{'}_2$을 얻고, 이를 다시 네트워크의 입력으로 넣는 과정을 $p_3=1$일 때까지 혹은 $i=N_{\text{max}}$일때까지 반복한다.
- 스케치를 샘플링하는건 중간의 VAE 때문에 deterministic하지 않다. 이런 randomness를 파라미터로 조절할 수 있는데, 온도와 같은 역할을 하는 파라미터 $\tau$를 도입해서 이 값이 크면 randomness가 커지도록 할 수 있다. $\tau$는 0과 1 사이의 값을 사용한다. (이 파라미터를 조절했을 때 그림의 변화는 Figure3로 확인할 수 있으며, 파란색은 낮은 온도, 붉은 색은 높은 온도일때 생성된 그림이다.)


### 3.3 Unconditional Generation

- 3.2에서 설명한 구조는 latent vector $z$에 conditional한 출력을 내놓는다. 이 구조에서 앞단의 인코더를 없애고, 디코더 RNN만으로 학습시킬 수도 있는데, 이는 latent 변수가 없는 autoregressive model이라 할 수 있다. 이렇게 바꿨을 때 최초 hidden state과 cell state은 0값들을 사용한다. 그리고 각 time-step에 RNN의 입력은 (training time) $S_{i-1}$과 (inference time) $S^{'}_{i-1}$이다. 이렇게 unconditional하게 생성한 그림이 Figure3에 나온다.

- 이때의 loss function은???

### 3.4 Training

- VAE에서 maximize하는 loss function은 아래와 같다. 즉 negative KL divergence와 MLE의 합인데, KL divergence를 최소화 하고, MLE(-reconstruction error)를 최대화 한다.(reconstruction error는 최소화)
$$\begin{align}
\mathcal{\tilde{L}}^B(\boldsymbol{\theta}, \boldsymbol{\phi}; \mathbf{x}^{(i)}) &= -  KL\big (q_\phi (\mathbf{z}^{(i, l)} \mid \mathbf{x}^{(i)}) \Vert p_\theta(\mathbf{z}^{(i, l)}) \big) + \frac{1}{L} \sum_{l=1}^L \log p_\boldsymbol{\theta}(\mathbf{x}^{(i)} \mid \mathbf{z}^{(i, l)}) \\
&=\frac{1}{2} \sum_{j=1}^J \bigg( 1 + \log( (\sigma_j^{(i)})^2 - (\mu_j^{(i)})^2) - (\sigma_j^{(i)})^2 \bigg) + \frac{1}{L} \sum_{l=1}^L \log p_\theta (\mathbf{x}^{(i)} \mid \mathbf{z}^{(i, l)} ) \\
~~~ \text{where} ~~ \mathbf{z}^{(i, l)} &= \boldsymbol{\mu}^{(i)} + \boldsymbol{\sigma}^{(l)} ~ \text{and} ~ \epsilon^{(l)} \sim \mathcal{N}(0, \mathbf{I}) 
\end{align}$$

- 이 논문에 최소화 하는 loss function은 VAE 논문에 등장하는 objective function, $\mathcal{\tilde{L}}^B(\boldsymbol{\theta}, \boldsymbol{\phi}; \mathbf{x}^{(i)})$의 부호를 바꾼 것이며          
KL divergence 항인 $K_{KL}$과       
reconstruction loss 항인 $L_R(=L_s + L_p)$을 합한 것이다.         
전체 loss를 최소화하므로 $K_{KL}$와 $L_R(=L_s + L_p)$ 모두 최소화 한다.

- $L_s$는 $\Delta x$와 $\Delta y$에 대한 loss인데, $N_S$이후(스케치가 끝나면) loss는 버린다. 반면 $L_p$는 $(p_1, p_2, p_3)$에대한 loss인데, $N_\max$까지 모든 loss를 사용한다. 이렇게 하면 $(p_1, p_2, p_3)$ 각각에 대한 loss에 weight을 달리하는 것 보다 강건하다.
- $L_{KL}$은 latent vector $z \sim p(z \mid x)$와 $z$의 prior $\mathcal{N}(0, 1)$의 차이를 의미한다.

- Q
 - reconstruction은 왜 joint prob 형태인가? 그리고 코딩할때는 단순히 mse인가? (작은 힌트: https://stats.stackexchange.com/questions/347378/variational-autoencoder-why-reconstruction-term-is-same-to-square-loss)

- Q
 - Figure4 좌측 그림에서 standalone decoder이라서 $L_{KL}$이 없는데 $w_{KL}$이 무슨 의미가 있지? -> 좌측 그림의 실선은 encoder를 포함한 모델이고, 점선은 standalone 모델이다.
 - Figure4 우측 그림에서 $L_{KL}$은 왜 점차 증가하지? Validation loss라서 그런가??? -> 원 논문의 SGVB(maximize)의 부호를 바꿨다. 그래서 학습될수록 KL은 커진다. 그리고 L_R은 likelihood이므로 작아진다.

- 전체 loss는 아래와 같이 $L_{KL}$에 가중치를 줄 수 있다. 즉 regularization 강도를 조절할 수 있다.
$$Loss = L_R + w_{KL} L_{KL}$$
 - $w_{KL}$ 값이 0에 가까워질수록 기본적인 autoencoder에 가까워진다.
 - unconditional generation에는 $L_{KL}$이 없다.
 

- Figure4(좌측): 점선은 standalone 모델로서 $L_{KL}$이 없는 모델이다. 그래서 encoder가 있는 모델 loss의 upper bound 쯤 된다. 

# 4. Experiments

- Q
 - 6페이지 첫머리에 breakdown of losses는 뭘까?
 - 6페이지 두번째 문단에 abstrct component???

### 4.1 Conditional Reconstruction

### 4.2 Latent Space Interpolation

### 4.3 Sketch Drawing Analogies

### 4.4 Predicting Different Endings of Incomplete Sketches

# 5. Applications and Future Work

# 6. Conclusion